In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
# import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Iso Year'] = Query['datetime'].dt.isocalendar().year
    Src['Iso Week'] = Query['datetime'].dt.isocalendar().week
    Src['Week Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Variat']   = Src['Close'] / Src['Open'] *100-100
    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF'], sort=0, group_keys=0)

    Calc[f'Variat +{1}']        =  Calc_by['Variat'].shift(-1)
    Calc[f'Variat +{1} > 0']    = (Calc[f'Variat +{1}'] > 0)                .astype(int)
    Calc[f'Variat +{1} < 0']    = (Calc[f'Variat +{1}'] < 0)                .astype(int)

    for x in [0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00]:
        Calc[f'Variat > +{x}%'] = (Calc['Variat'] > +x/100)                 .astype(int)
        Calc[f'Variat < -{x}%'] = (Calc['Variat'] < -x/100)                 .astype(int)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [ ]:
def STATS_UP(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=1):
        Df = Calc[Calc[f'Variat > +{x}%']==1].reset_index(drop=1)

        pipe.append({
            'Variat > %':  x,
            'Variat +1 > 0': np.round(Df['Variat +1 > 0'].mean() *100, 1), 
            'Variat +1 < 0': np.round(Df['Variat +1 < 0'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS_UP(Calc)

,Variat > %,Variat +1 > 0,Variat +1 < 0
0,5.0,49.6,50.2
1,4.5,49.9,49.9
2,4.0,49.9,49.9
3,3.5,49.9,49.9
4,3.0,49.8,50.0
5,2.5,49.8,50.0
6,2.0,49.8,50.1
7,1.5,49.8,50.0
8,1.0,49.8,50.1
9,0.5,49.6,50.2


In [56]:
def STATS_DOWN(Calc):
    pipe = []
    for x in sorted([0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00], reverse=0):
        Df = Calc[Calc[f'Variat < -{x}%']==1].reset_index(drop=1)

        pipe.append({
            'Variat < %':  -x,
            'Variat +1 > 0': np.round(Df['Variat +1 > 0'].mean() *100, 1), 
            'Variat +1 < 0': np.round(Df['Variat +1 < 0'].mean() *100, 1), 
        })
    pass
    return pd.DataFrame(pipe)

STATS_DOWN(Calc)

,Variat < %,Variat +1 > 0,Variat +1 < 0
0,-0.5,49.5,50.3
1,-1.0,49.5,50.3
2,-1.5,49.8,50.1
3,-2.0,49.8,50.1
4,-2.5,49.8,50.0
5,-3.0,49.8,50.0
6,-3.5,49.8,50.0
7,-4.0,49.8,50.0
8,-4.5,49.8,50.0
9,-5.0,49.9,49.9
